# **Distribución % de créditos por segmento (trimestre)**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

In [3]:
df_financiamientos_2022_puebla= df_financiamientos_2022_puebla[['id','mes','monto']]
df_financiamientos_2023_puebla= df_financiamientos_2023_puebla[['id','mes','monto']]
df_financiamientos_2024_puebla= df_financiamientos_2024_puebla[['id','mes','monto']]

## 2022

In [9]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2022_puebla['trimestre'] = df_financiamientos_2022_puebla['mes'].apply(asignar_trimestre)
df_financiamientos_2022_puebla

,id,mes,monto,trimestre,segmento
0,1,1,21093.00,1,E
1,2,1,34318.00,1,E
2,3,1,43431.00,1,E
3,4,1,51998.00,1,E
4,5,1,94648.96,1,E
...,...,...,...,...,...
3099,3100,12,0.00,4,E
3100,3101,12,0.00,4,E
3101,3102,12,0.00,4,E
3102,3103,12,0.00,4,E


In [10]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_puebla['segmento'] = df_financiamientos_2022_puebla['monto'].apply(asignar_segmento)
df_financiamientos_2022_puebla.head()

,id,mes,monto,trimestre,segmento
0,1,1,21093.00,1,E
1,2,1,34318.00,1,E
2,3,1,43431.00,1,E
3,4,1,51998.00,1,E
4,5,1,94648.96,1,E


In [11]:
trimestres_credit_2022_pachuca = df_financiamientos_2022_puebla[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
3099,4,E
3100,4,E
3101,4,E
3102,4,E


In [12]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2022_puebla)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2022_puebla, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  3104
trimestre          1          2          3          4
segmento                                             
A           0.579897   0.966495   0.740979   0.676546
B           1.192010   1.417526   1.288660   1.643041
C           2.351804   2.641753   2.384021   2.480670
D           5.315722   5.025773   5.122423   5.122423
E          15.077320  13.434278  14.143041  12.274485
S           1.030928   1.739691   1.643041   1.707474


In [14]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
#porcentajes = tabla_porcentajes[trimestre]
fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_%_2022',  carpeta='assets/graficas')
fig.show()

## 2023

In [15]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2023_puebla['trimestre'] = df_financiamientos_2023_puebla['mes'].apply(asignar_trimestre)
df_financiamientos_2023_puebla

,id,mes,monto,trimestre
0,1,1,19817.75,1
1,2,1,0.00,1
2,3,1,0.00,1
3,4,1,33908.00,1
4,5,1,277298.00,1
...,...,...,...,...
10832,10833,12,67786.64,4
10833,10834,12,0.00,4
10834,10835,12,0.00,4
10835,10836,12,0.00,4


In [16]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_puebla['segmento'] = df_financiamientos_2023_puebla['monto'].apply(asignar_segmento)
df_financiamientos_2023_puebla.head()

,id,mes,monto,trimestre,segmento
0,1,1,19817.75,1,E
1,2,1,0.00,1,E
2,3,1,0.00,1,E
3,4,1,33908.00,1,E
4,5,1,277298.00,1,E


In [17]:
df_financiamientos_2023_puebla = df_financiamientos_2023_puebla[['trimestre','segmento']]
df_financiamientos_2023_puebla

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
10832,4,E
10833,4,E
10834,4,E
10835,4,E


In [18]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2023_puebla)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2023_puebla, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  10837
trimestre          1          2          3          4
segmento                                             
A           0.424472   0.590569   0.525976   0.756667
B           0.904309   0.941220   1.153456   1.245732
C           1.993172   2.030082   2.186952   2.371505
D           4.189351   4.881425   4.992157   5.213620
E          13.869152  15.364031  13.795331  16.775860
S           1.144228   1.476423   1.504106   1.670204


In [20]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
#porcentajes = tabla_porcentajes[trimestre]

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_%_2023',  carpeta='assets/graficas')
fig.show()

## 2024

In [21]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2024_puebla['trimestre'] = df_financiamientos_2024_puebla['mes'].apply(asignar_trimestre)
df_financiamientos_2024_puebla

,id,mes,monto,trimestre
0,1,1,8574319.0,1
1,2,1,1633800.0,1
2,3,1,0.0,1
3,4,1,0.0,1
4,5,1,0.0,1
...,...,...,...,...
2492,2493,3,0.0,1
2493,2494,3,0.0,1
2494,2495,3,0.0,1
2495,2496,3,0.0,1


In [22]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_puebla['segmento'] = df_financiamientos_2024_puebla['monto'].apply(asignar_segmento)
df_financiamientos_2024_puebla.head()

,id,mes,monto,trimestre,segmento
0,1,1,8574319.0,1,S
1,2,1,1633800.0,1,D
2,3,1,0.0,1,E
3,4,1,0.0,1,E
4,5,1,0.0,1,E


In [23]:
df_financiamientos_2024_puebla = df_financiamientos_2024_puebla[['trimestre','segmento']]
df_financiamientos_2024_puebla

,trimestre,segmento
0,1,S
1,1,D
2,1,E
3,1,E
4,1,E
...,...,...
2492,1,E
2493,1,E
2494,1,E
2495,1,E


In [24]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2024_puebla)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2024_puebla, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  2497
trimestre          1
segmento            
A           2.442932
B           4.245094
C           9.050861
D          19.383260
E          59.191029
S           5.686824


In [25]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
#porcentajes = tabla_porcentajes[trimestre]

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_%_2024',  carpeta='assets/graficas')
fig.show()